In [11]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras import optimizers
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from keras.preprocessing import image
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
batch_size = 128
num_classes = 120
epochs = 10

# input image dimensions
img_x, img_y = 224, 224

# load the MNIST data set, which already splits into train and test sets for us


In [12]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(img_x, img_y))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [13]:
pth = "/home/ubuntuuser/DogBreedClassification/Notebooks/train"
label_df = pd.read_csv("/home/ubuntuuser/DogBreedClassification/Notebooks/labels.csv")
labels = list(label_df.breed)
train_paths = []
for root, dirs, files in os.walk(pth):
    for file in files:
        train_paths.append(os.path.join(root,file))
train_data = paths_to_tensor(train_paths)


 27%|██▋       | 2718/10222 [00:16<00:45, 166.07it/s]


 53%|█████▎    | 5466/10222 [00:31<00:27, 171.35it/s]


 80%|███████▉  | 8169/10222 [00:47<00:11, 171.55it/s]


100%|██████████| 10222/10222 [00:59<00:00, 171.08it/s]

MemoryError: 

Encode the labels

In [102]:
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)

In [106]:
X_train, X_test, y_train, y_test = train_test_split(train_data, encoded_Y, test_size=0.33, random_state=42)

In [126]:
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)

In [109]:
input_shape = (img_x, img_y, 3)

In [110]:
# convert the data to the right type
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('x_train shape:', X_train.shape)
print("input shape", input_shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (6848, 224, 224, 3)
input shape (224, 224, 3)
6848 train samples
3374 test samples


In [121]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224,224,3), data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [129]:
history = model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

Train on 6848 samples, validate on 3374 samples
Epoch 1/10
6848/6848 [==============================] - 32938s 5s/step - loss: 15.9863 - acc: 0.0082 - val_loss: 15.9461 - val_acc: 0.0107
Epoch 2/10
6848/6848 [==============================] - 510s 74ms/step - loss: 15.9804 - acc: 0.0085 - val_loss: 15.9557 - val_acc: 0.0101
Epoch 3/10
6848/6848 [==============================] - 498s 73ms/step - loss: 15.9606 - acc: 0.0096 - val_loss: 15.9652 - val_acc: 0.0095
Epoch 4/10
6848/6848 [==============================] - 528s 77ms/step - loss: 15.9533 - acc: 0.0102 - val_loss: 15.9652 - val_acc: 0.0095
Epoch 5/10
6848/6848 [==============================] - 471s 69ms/step - loss: 15.9533 - acc: 0.0102 - val_loss: 15.9652 - val_acc: 0.0095
Epoch 6/10
4480/6848 [==================>...........] - ETA: 3:13 - loss: 15.9382 - acc: 0.0112

KeyboardInterrupt: 